In [1]:
import configparser
import os
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import udf, col

In [2]:
config = configparser.ConfigParser()
config.read('dl.cfg')

['dl.cfg']

In [3]:
os.environ['AWS_ACCESS_KEY_ID']=config.get('AWS','AWS_ACCESS_KEY_ID')
os.environ['AWS_SECRET_ACCESS_KEY']=config.get('AWS','AWS_SECRET_ACCESS_KEY')
os.environ['AWS_REGION']=config.get('AWS','REGION')

In [4]:
spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate() 

In [5]:
# read song data file
output_data = config.get('LOCAL_DATA', 'OUTPUT')
print('output_data : ' + output_data)

# read songs for 1994
song_data_df = spark.read.parquet(os.path.join(output_data, 'songs/year=1994/artist_id=*/*.parquet'))

print ('Number of song rows : ' + str(song_data_df.count()))

song_data_df.limit(5).toPandas()

output_data : data/output/
Number of song rows : 2


,song_id,title,duration
0,SOFFKZS12AB017F194,A Higher Place (Album Version),236.17261
1,SONWXQJ12A8C134D94,The Ballad Of Sleeping Beauty,305.16200


In [6]:
song_data_df = spark.read.parquet(os.path.join(output_data, 'songs'))
print ('Number of song rows : ' + str(song_data_df.count()))

# display all columns of songs
song_data_df.limit(10).orderBy('song_id').toPandas()                                 

Number of song rows : 71


,song_id,title,duration,year,artist_id
0,SOAOIBZ12AB01815BE,I Hold Your Hand In Mine [Live At Royal Albert...,43.36281,2000,ARPBNLO1187FB3D52F
1,SOBAYLL12A8C138AF9,Sono andati? Fingevo di dormire,511.16363,0,ARDR4AC1187FB371A1
2,SOBBUGU12A8C13E95D,Setting Fire to Sleeping Giants,207.77751,2004,ARMAC4T1187FB3FA4C
3,SODREIN12A58A7F2E5,A Whiter Shade Of Pale (Live @ Fillmore West),326.00771,0,ARLTWXK1187FB5A3F8
4,SONYPOM12A8C13B2D7,I Think My Wife Is Running Around On Me (Taco ...,186.48771,2005,ARDNS031187B9924F0
5,SOOLYAZ12A6701F4A6,Laws Patrolling (Album Version),173.66159,0,AREBBGV1187FB523D2
6,SOPEGZN12AB0181B3D,Get Your Head Stuck On Your Neck,45.66159,0,AREDL271187FB40F44
7,SOUDSGM12AC9618304,Insatiable (Instrumental Version),266.39628,0,ARNTLGG11E2835DDB9
8,SOWQTQZ12A58A7B63E,Streets On Fire (Explicit Album Version),279.97995,0,ARPFHN61187FB575F6
9,SOYMRWW12A6D4FAB14,The Moon And I (Ordinary Day Album Version),267.70240,0,ARKFYS91187B98E58F


In [7]:

####################### Read parquet file for songplays ##################
output_data = config.get('LOCAL_DATA', 'OUTPUT')
print('output_data : ' + output_data)

songplays_data_df = spark.read.parquet(os.path.join(output_data, 'songplays'))

print ('Number of songPlays Table rows : ' + str(songplays_data_df.count()))                                                   

songplays_data_df = spark.read.parquet(os.path.join(output_data, 'songplays/year=2018/month=11/*.parquet'))

songplays_data_df.limit(5).toPandas()

output_data : data/output/
Number of songPlays Table rows : 4


,songplay_id,start_time,user_id,level,song_id,artist_id,session_id,location,user_agent
0,403726925824,2018-11-21 21:56:47.796,15,paid,SOZCTXZ12AB0182364,AR5KOSW1187FB35FF4,818,"Chicago-Naperville-Elgin, IL-IN-WI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5..."
1,128849018880,2018-11-14 05:06:03.796,10,free,SOGDBUF12A8C140FAA,AR558FS1187FB45658,484,"Washington-Arlington-Alexandria, DC-VA-MD-WV","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
2,1529008357376,2018-11-27 22:35:59.796,80,paid,SOGDBUF12A8C140FAA,AR558FS1187FB45658,992,"Portland-South Portland, ME","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
3,566935683072,2018-11-19 09:14:20.796,24,paid,SOGDBUF12A8C140FAA,AR558FS1187FB45658,672,"Lake Havasu City-Kingman, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK..."


In [8]:
########################## Read artists table ######################################### 
artists_data_df = spark.read.parquet(os.path.join(output_data, 'artists/*.parquet'))
print('Number of artists Table rows : ' + str(artists_data_df.count()))

artists_data_df.limit(5).toPandas()

Number of artists Table rows : 69


,artist_id,name,location,latitude,longitude
0,ARNF6401187FB57032,Sophie B. Hawkins,"New York, NY [Manhattan]",40.79086,-73.96644
1,AROUOZZ1187B9ABE51,Willie Bobo,"New York, NY [Spanish Harlem]",40.79195,-73.94512
2,AREBBGV1187FB523D2,Mike Jones (Featuring CJ_ Mello & Lil' Bran),"Houston, TX",NaN,NaN
3,ARD842G1187B997376,Blue Rodeo,"Toronto, Ontario, Canada",43.64856,-79.38533
4,AR9AWNF1187B9AB0B4,Kenny G featuring Daryl Hall,"Seattle, Washington USA",NaN,NaN


In [9]:
################################################ Read Time table ############################################# 
print ('All Time columns')
time_df = spark.read.parquet(os.path.join(output_data, 'time'))
time_df.limit(5).toPandas()

All Time columns


,start_time,hour,day,week,weekday,year,month
0,2018-11-15 00:30:26.796,0,15,46,5,2018,11
1,2018-11-15 00:41:21.796,0,15,46,5,2018,11
2,2018-11-15 00:45:41.796,0,15,46,5,2018,11
3,2018-11-15 03:44:09.796,3,15,46,5,2018,11
4,2018-11-15 05:48:55.796,5,15,46,5,2018,11


In [10]:
print ('Time columns for year 2018')
time_data_df = spark.read.parquet(os.path.join(output_data, 'time/year=2018/month=*/*.parquet'))
print ('Number of Time Table rows : ' + str(time_data_df.count()))

time_data_df.limit(5).toPandas()

Time columns for year 2018
Number of Time Table rows : 6820


,start_time,hour,day,week,weekday
0,2018-11-15 00:30:26.796,0,15,46,5
1,2018-11-15 00:41:21.796,0,15,46,5
2,2018-11-15 00:45:41.796,0,15,46,5
3,2018-11-15 03:44:09.796,3,15,46,5
4,2018-11-15 05:48:55.796,5,15,46,5


In [11]:
################################################## Read Users table ###################################################
users_data_df = spark.read.parquet(os.path.join(output_data, 'users/*.parquet'))
print ('Number of Users Table rows : ' + str(users_data_df.count()))

# display paid and free users by gender
users_data_df.groupby('level', 'gender').count().show()

Number of Users Table rows : 104
+-----+------+-----+
|level|gender|count|
+-----+------+-----+
| free|     M|   37|
| paid|     M|    7|
| paid|     F|   15|
| free|     F|   45|
+-----+------+-----+



In [12]:
################################# to read songplays table  ############################################
#df_songplays = sqlContext.read.parquet('data/output/songplays')

output_data = config.get('LOCAL_DATA', 'OUTPUT')
print('output_data : ' + output_data)

songplays_data_df = spark.read.parquet(os.path.join(output_data, 'songplays/year=2018/month=*/*.parquet'))

print ('Number of SongPlays Table rows : ' + str(songplays_data_df.count()))

songplays_data_df.limit(10).toPandas()

output_data : data/output/
Number of SongPlays Table rows : 4


,songplay_id,start_time,user_id,level,song_id,artist_id,session_id,location,user_agent
0,403726925824,2018-11-21 21:56:47.796,15,paid,SOZCTXZ12AB0182364,AR5KOSW1187FB35FF4,818,"Chicago-Naperville-Elgin, IL-IN-WI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5..."
1,128849018880,2018-11-14 05:06:03.796,10,free,SOGDBUF12A8C140FAA,AR558FS1187FB45658,484,"Washington-Arlington-Alexandria, DC-VA-MD-WV","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
2,1529008357376,2018-11-27 22:35:59.796,80,paid,SOGDBUF12A8C140FAA,AR558FS1187FB45658,992,"Portland-South Portland, ME","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
3,566935683072,2018-11-19 09:14:20.796,24,paid,SOGDBUF12A8C140FAA,AR558FS1187FB45658,672,"Lake Havasu City-Kingman, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK..."


In [13]:
songplays_data_df.groupBy('song_id').count().show()
print ('Display song_ids in songplays for the year 2018')
pdsplay = songplays_data_df.groupBy('song_id').count().toPandas()
print (pdsplay)

+------------------+-----+
|           song_id|count|
+------------------+-----+
|SOZCTXZ12AB0182364|    1|
|SOGDBUF12A8C140FAA|    3|
+------------------+-----+

Display song_ids in songplays for the year 2018
              song_id  count
0  SOZCTXZ12AB0182364      1
1  SOGDBUF12A8C140FAA      3


In [14]:
#print ('Display song_id associated with free and paid levels for the year 2018')
#songplays_data_df.groupBy('song_id', 'level').count().show()

In [15]:
pdsplay1 = songplays_data_df.groupBy('song_id', 'level').count().toPandas()
print (pdsplay1)

pdsplay1.sort_values(by='song_id', ascending=True, inplace=False, na_position='last')

              song_id level  count
0  SOGDBUF12A8C140FAA  paid      2
1  SOZCTXZ12AB0182364  paid      1
2  SOGDBUF12A8C140FAA  free      1


,song_id,level,count
0,SOGDBUF12A8C140FAA,paid,2
2,SOGDBUF12A8C140FAA,free,1
1,SOZCTXZ12AB0182364,paid,1
